In [5]:
class FeatureSelector():
    def __init__(self, estimator,
                       metric,\
                       larger_is_better,\
                       cv,
                       use_values,\
                       use_recursion,
                       fill_na,\
                       show_progress, 
                       early_stopping = None):
        '''
        Инициализирует модель для отбора признаков
        
        Параметры:
            1) estimator - модель
            2) metric - метрика качества (названия метрик sklearn + может быть кастомная)
            3) larger_is_better - критерий оптимизации (чем больше, тем лучше)
            4) cv - схема валидации
            5) use_values - индексы столбцов, в которых требуется отобрать значения
            6) use_recursion - использовать рекурсию в отборе
            7) fill_na - значение, которым заполняются np.nan
            8) show_progress - печатать результаты валидации
            9) early_stopping - число итераций без улучшения метрики для ранней остановки отбора
        Возвращает:
            1) fit - производит отбор признаков
            2) transform - оставляет отобранные признаки
            3) return_self - возвращает 
                - best_features - отобранные признаки(список)
                - D_best_features - отобранные значения признаков (словарь: {признак:значения})
                - best_score - лучшее значение метрики
        '''
        self.estimator = estimator
        self.metric = metric
        self.cv = cv
        self.use_values = use_values        
        self.use_recursion = use_recursion
        self.show_progress = show_progress
        self.early_stopping = early_stopping
        self.fill_na = fill_na
        self.larger_is_better = larger_is_better
    def fit(self, X, Y):        
        flag = isinstance(X[:, 0], csc_matrix)
        # список с результатами валидации
        column_value_score = []
        # проходим по признакам
        for i in tqdm_notebook(range(X.shape[1])):
            # если формат матрицы признаков == csc_matrix
            if flag:
                # выбираем столбец, преобразуем
                ser = pd.DataFrame(X[:, i].todense())[0].values.flatten()
            # если формат != csc_matrix
            else:
                # выбираем столбец
                ser = X[:, i]        
            # если столбец в списке с проверкой значений 
            if self.use_values is not None:                
                if i in self.use_values:
                    # уникальные значения столбца
                    unique_values = np.unique(ser)  
                    # валидируем каждое значение
                    for val in unique_values:
                        _x = np.int32(ser==val).reshape(-1,1)
                        column_value_score.append((i, val,\
                                                   cross_val_score(self.estimator,\
                                                                   _x, Y,\
                                                                   scoring = self.metric,\
                                                                   cv = self.cv).mean()))
                else: 
                    # валидируем столбец
                    column_value_score.append((i, None,\
                                               cross_val_score(self.estimator,\
                                                               _x, Y,\
                                                               scoring = self.metric,\
                                                               cv = self.cv).mean()))
            else:
                # валидируем столбец
                    column_value_score.append((i, None,\
                                               cross_val_score(self.estimator,\
                                                               ser.reshape(-1,1), Y,\
                                                               scoring = self.metric,\
                                                               cv = self.cv).mean()))
                

        # признаки и значения признаков в порядке убывания валидации
        order = np.array(sorted(column_value_score, key = lambda x: x[-1], reverse = True))[:, :2]             
        # список лучших признаков
        best_features = []
        # словарь лучших значений признаков
        D_best_features = defaultdict(list)
        # список с признаками, не давшими прироста
        to_drop = []
        
        # лучшее значение метрики
        if self.larger_is_better:
            best_score = 0
        else:
            best_score = np.inf            
        counter = 0
        # проходим по признакам и значениям признаков в порядке убывания валидации
        for feature, feature_value in tqdm_notebook(order):   

            # добавляем текущие признаки/значения
            if feature_value is None:
                best_features.append(feature)               
            else:
                D_best_features[feature].append(feature_value)

            # обновляем матрицы
            L = []
            for k, v in D_best_features.items():
                if isinstance(X[:, k], csc_matrix):
                    L.append(pd.DataFrame(X[:, k].tocsc().todense())[0].apply(lambda x: x if x in v else self.fill_na))
                else:
                    L.append(pd.Series(X[:, k].flatten()).apply(lambda x: x if x in v else self.fill_na))

            if flag:
                if (len(best_features)>0) & (len(L)>0):
                    _X = csc_matrix(hstack([X[:, best_features], csc_matrix(np.column_stack(L)) ]))
                elif (len(best_features)==0) & (len(L)>0):
                    _X = csc_matrix(np.column_stack(L))
                elif (len(best_features)>0) & (len(L)==0):
                    _X = csc_matrix(X[:, best_features])                    
                        
            else:
                if (len(best_features)>0) & (len(L)>0):
                    _X = np.column_stack([X[:, best_features], np.column_stack(L)])
                elif (len(best_features)==0) & (len(L)>0):
                    _X = np.column_stack(L)
                elif (len(best_features)>0) & (len(L)==0):
                    _X = X[:, best_features] 
            # считаем валидацию    
            current_score = cross_val_score(self.estimator, _X, Y, scoring = self.metric, cv = self.cv).mean()
            # если метрика улучшилась
            if self.larger_is_better:
                if current_score>best_score:
                    # обновляем лучшую метрику
                    best_score = current_score
                    counter = 0
                    # печатаем 
                    if self.show_progress:
                        print('new best_score = {}'.format(best_score))
                # если метрика не улучшилась
                else: 
                    counter+=1
                    # удаляем признак/значение
                    if feature_value is None:
                        best_features = [val for val in best_features if val != feature]
                        to_drop.append((feature, None))
                    else:
                        D_best_features[feature] = [val for val in D_best_features[feature] if val != feature_value]    
                        to_drop.append((feature, feature_value))
                    if counter == self.early_stopping:
                        break
            else:
                if current_score<best_score:
                    # обновляем лучшую метрику
                    best_score = current_score
                    counter = 0
                    # печатаем 
                    if self.show_progress:
                        print('new best_score = {}'.format(best_score))
                    # если метрика не улучшилась
                else: 
                    counter+=1
                    # удаляем признак/значение
                    if feature_value is None:
                        best_features = [val for val in best_features if val != feature]
                        to_drop.append((feature, None))
                    else:
                        D_best_features[feature] = [val for val in D_best_features[feature] if val != feature_value]    
                        to_drop.append((feature, feature_value))
                    if counter == self.early_stopping:
                        break

        if self.use_recursion:
            # запускаем бесконечный цикл
            while True:
                # списки лучших признаков до и после
                to_drop_before = to_drop
                to_drop_after = []
                # проходим по признакам и значениям признаков в порядке убывания валидации
                for feature, feature_value in tqdm_notebook(to_drop_before):   
                    # добавляем текущие признаки/значения
                    if feature_value is None:
                        best_features.append(feature)               
                    else:
                        D_best_features[feature].append(feature_value)

                    # обновляем матрицы
                    L = []
                    for k, v in D_best_features.items():
                        if isinstance(X[:, k], csc_matrix):
                            L.append(pd.DataFrame(X[:, k].tocsc().todense())[0].apply(lambda x: x if x in v else self.fill_na))
                        else:
                            L.append(pd.Series(X[:, k].flatten()).apply(lambda x: x if x in v else self.fill_na))

                    if flag:
                        if (len(best_features)>0) & (len(L)>0):
                            _X = csc_matrix(hstack([X[:, best_features], csc_matrix(np.column_stack(L)) ]))
                        elif (len(best_features)==0) & (len(L)>0):
                            _X = csc_matrix(np.column_stack(L))
                        elif (len(best_features)>0) & (len(L)==0):
                            _X = csc_matrix(X[:, best_features])                    

                    else:
                        if (len(best_features)>0) & (len(L)>0):
                            _X = np.column_stack([X[:, best_features], np.column_stack(L)])
                        elif (len(best_features)==0) & (len(L)>0):
                            _X = np.column_stack(L)
                        elif (len(best_features)>0) & (len(L)==0):
                            _X = X[:, best_features] 

                    # считаем валидацию    
                    current_score = cross_val_score(self.estimator, _X, Y, scoring = self.metric, cv = self.cv).mean()
                    
                    
                    
                    
                    
                    
                    if self.larger_is_better:
                        if current_score>best_score:
                            # обновляем лучшую метрику
                            best_score = current_score
                            counter = 0
                            # печатаем 
                            if self.show_progress:
                                print('new best_score = {}'.format(best_score))
                            # если метрика не улучшилась
                        else: 
                            # удаляем признак/значение
                            if feature_value is None:
                                best_features = [val for val in best_features if val != feature]
                                to_drop_after.append((feature, None))
                            else:
                                D_best_features[feature] = [val for val in D_best_features[feature] if val != feature_value]    
                                to_drop_after.append((feature, feature_value))
                    else:
                        if current_score<best_score:
                            # обновляем лучшую метрику
                            best_score = current_score
                            counter = 0
                            # печатаем 
                            if self.show_progress:
                                print('new best_score = {}'.format(best_score))
                        else: 
                            # удаляем признак/значение
                            if feature_value is None:
                                best_features = [val for val in best_features if val != feature]
                                to_drop_after.append((feature, None))
                            else:
                                D_best_features[feature] = [val for val in D_best_features[feature] if val != feature_value]    
                                to_drop_after.append((feature, feature_value))
                    

                # если списки одинаковые, останавливаем отбор
                if len(to_drop_after) == len(to_drop_before):
                    break
                # если разные - обновляем списки до и после
                else:
                    to_drop_before = to_drop_after
                    to_drop_after = []
                    
        self.best_features = best_features
        self.D_best_features = D_best_features
        self.best_score =best_score
        self.flag = flag
    def transform(self, X):
              
        if len(self.best_features) !=0:
            x1 = X[:, self.best_features]
        else:
            x1 = None
        if len(list(self.D_best_features.keys())) !=0:
            L=[]
            for k, v in self.D_best_features.items():
                if self.flag:
                    L.append(pd.DataFrame(X[:, k].tocsc().todense())[0].apply(lambda x: x if x in v else self.fill_na))                    
                else:
                    L.append(pd.Series(X[:, k].flatten()).apply(lambda x: x if x in v else self.fill_na))
            x2 = np.column_stack(L)
        else:
            x2 = None
            
        if (x1 is not None) & (x2 is not None):
            if self.flag: 
                _X = csc_matrix(hstack([x1, x2]))
            else:
                _X = np.column_stack([x1, x2])
                
        if (x1 is not None) & (x2 is None):
            _X = x1
        if (x1 is None) & (x2 is not None):
            if self.flag:
                _X = csc_matrix(x2)
            else:
                _X = x2
        return _X     
        
    def return_self(self):
        return self